In [1]:
import numpy as np 
import pandas as pd 
import csv 
import os 

import torch 


torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [2]:
path = os.getcwd()
DATA_PATH = "/".join(p for p in path.split(os.path.sep)[:-1]) + "/data/p1ch4/"
CSV_PATH = DATA_PATH + "bike-sharing-dataset/hour-fixed.csv"

In [4]:
bikes_numpy = np.loadtxt(CSV_PATH, dtype=np.float32, delimiter=',', skiprows=1, 
                    converters={1: lambda x: float(x[8:10])})

bikes = torch.from_numpy(bikes_numpy)
bikes, bikes.shape

(tensor([[1.00e+00, 1.00e+00,  ..., 1.30e+01, 1.60e+01],
         [2.00e+00, 1.00e+00,  ..., 3.20e+01, 4.00e+01],
         ...,
         [1.74e+04, 3.10e+01,  ..., 4.80e+01, 6.10e+01],
         [1.74e+04, 3.10e+01,  ..., 3.70e+01, 4.90e+01]]),
 torch.Size([17520, 17]))